In [1]:
!pip install transformers datasets faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 75.7 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login
login(new_session=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': " I am an artificial intelligence designed to assist with information and answer questions to the best of my ability. I don't have the ability to have a personality or emotions, but I am programmed to process and analyze data in order to provide accurate and helpful responses. I am here to make your life easier by providing information and answering your questions. How can I help you today?"}]}]

In [9]:


# LLM'e görev verecek şekilde sistem mesajı ve kullanıcı mesajını birleştiriyoruz
system_prompt = "Sen bir yardımcı agentsın. Kullanıcının sorusunu açık ve detaylı şekilde cevapla."
user_prompt = "Ankara'da hava nasıl?"

# Model mesaj formatını simüle ediyoruz (Chat formatına benzer)
messages = f"<s>[INST] {system_prompt}\n{user_prompt} [/INST]"

# Çıktı üret
output = pipe(messages, max_new_tokens=200, do_sample=True, temperature=0.7)

# Sonucu yazdır
print(output[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] Sen bir yardımcı agentsın. Kullanıcının sorusunu açık ve detaylı şekilde cevapla.
Ankara'da hava nasıl? [/INST] Ankara, Türkiye'nin batısındaki bir şehir olduğu için orta yıllarda asansörlü bir aklima sahip bir şehir olabilir. Ancak, Ankara hava şartları hafif bir zaman çok değişebilir. İşte Ankara'da hava durumunu ara ederek lütfen aşağıdaki bağlantıyı kullanarak en güncel bilgiler alabilirsiniz:

https://www.weather.com/tr-TR/weather/today/ankara+1388476

Bu sayfada, Ankara'da geçen günün hava durum


In [6]:
def get_current_weather(location: str) -> str:
    if location.lower() == "ankara":
        return "Ankara'da hava 26°C ve güneşli."
    elif location.lower() == "istanbul":
        return "İstanbul'da hava 23°C ve parçalı bulutlu."
    else:
        return f"{location} için hava durumu verisi bulunamadı."


In [8]:
user_input = "Ankara'da hava nasıl?"

# Tool çağır
tool_output = get_current_weather("Ankara")

# Prompt oluştur
prompt = f"""
[INST] Kullanıcı sordu: "{user_input}"
Tool'dan alınan bilgi: "{tool_output}"
Şimdi bu bilgiyi kullanıcıya açık ve kibar şekilde anlat. [/INST]
"""

# LLM'e ver
response = pipe(prompt, max_new_tokens=150, do_sample=True)[0]['generated_text']
print(response)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[INST] Kullanıcı sordu: "Ankara'da hava nasıl?"
Tool'dan alınan bilgi: "Ankara'da hava 26°C ve güneşli."
Şimdi bu bilgiyi kullanıcıya açık ve kibar şekilde anlat. [/INST]
"Bu anlar Ankara'da, çok iyi bulunuyorsunuz! Hava şu anda 26 derecelik sıcaklık taşıyor. Güneşi de çok görüyorsunuz, böylece güneşin sıcaklığı artık Ankara'da nasıl değil mi? Şimdiye kadar iyi bir gün geçiyorsunuz!"


In [10]:
def get_current_weather(location: str) -> str:
    if location.lower() == "ankara":
        return "Ankara'da hava 26°C ve güneşli."
    elif location.lower() == "istanbul":
        return "İstanbul'da hava 23°C ve parçalı bulutlu."
    else:
        return f"{location} için hava durumu verisi bulunamadı."

def multiply_numbers(a: int, b: int) -> str:
    return f"{a} ile {b} çarpımı: {a * b}"


In [13]:
user_input = "12 ile 8'i çarpar mısın?"

# Basit karar mantığı:
if "hava" in user_input.lower():
    tool_output = get_current_weather("Ankara")
elif "çarp" in user_input.lower():
    tool_output = multiply_numbers(12, 8)
else:
    tool_output = "Tool gerekmedi."

# Prompt:
prompt = f"""
[INST] Kullanıcıdan gelen mesaj: "{user_input}"
Tool sonucu: "{tool_output}"
Kullanıcıya detaylı ama sade bir açıklama yap. [/INST]
"""

from transformers import pipeline

response = pipe(prompt, max_new_tokens=500, do_sample=True)[0]['generated_text']
print(response)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[INST] Kullanıcıdan gelen mesaj: "12 ile 8'i çarpar mısın?"
Tool sonucu: "12 ile 8 çarpımı: 96"
Kullanıcıya detaylı ama sade bir açıklama yap. [/INST]
"Evet, 12 ile 8'in çarpımı 96'dır."

Veya,

"12'e 8 ekledik ve sonucu aldık. Sonucu 96'dır."

Veya,

"12'e 8 katmış olduk, sonucu 96 aldık."

Veya,

"İki sayı 12 ve 8 oldu. Bu sayıların çarpımını buldum ve sonucu 96 oldu."


In [31]:
docs = [
    "Türkiye'nin en büyük gölü Van Gölü'dür. Yüzölçümü yaklaşık 3,755 km²'dir.",
    "Ankara Üniversitesi 1946 yılında kurulmuştur ve Türkiye'nin köklü üniversitelerinden biridir.",
    "2024 yılı MTA raporuna göre, Türkiye'de 5 milyon ton kömür üretimi yapılmıştır."
]



In [33]:
from transformers import pipeline


# Soru
query = "Türkiye'de 2024 yılında ne kadar kömür üretildi?"
llm_response = pipe(f"<s>[INST] {query} [/INST]", max_new_tokens=100)
print("🔵 LLM Cevabı:\n", llm_response[0]['generated_text'])




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🔵 LLM Cevabı:
 <s>[INST] Türkiye'de 2024 yılında ne kadar kömür üretildi? [/INST] According to the Turkish Statistical Institute (TurkStat), Turkey produced approximately 46 million tonnes of coal in 2024. However, I could not find the exact data as the year 2024 has not yet come, so this is an estimate based on the trend of coal production in recent years. For the most accurate information, please check the TurkStat website or contact them directly.


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 🔍 En alakalı dokümanı bul
vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform(docs)
query_vector = vectorizer.transform([query])
similarities = cosine_similarity(query_vector, doc_vectors)
most_relevant_doc = docs[similarities.argmax()]

# 📜 LLM'e prompt hazırlama
rag_prompt = f"""
<s>[INST]
Soru: {query}

İlgili doküman:
{most_relevant_doc}

Cevapla: [/INST]
"""

rag_response = pipe(rag_prompt, max_new_tokens=100)
print("🟢 RAG Destekli Cevap:\n", rag_response[0]['generated_text'])



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🟢 RAG Destekli Cevap:
 
<s>[INST]
Soru: Türkiye'de 2024 yılında ne kadar kömür üretildi?

İlgili doküman:
2024 yılı MTA raporuna göre, Türkiye'de 5 milyon ton kömür üretimi yapılmıştır.

Cevapla: [/INST]
2024 yılında Türkiye'de 5 milyon ton kömür üretilmiştir. (According to the MTA report for 2024, 5 million tons of coal were produced in Turkey.)
